In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import yaml
from glob import glob
import rawpy
import zivid
import os
import open3d as o3d
from tqdm import trange
from calib import load_zdf_rgb, load_fuji_rgb, convert_16bit_to_8bit

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
calib_path = "../data/20221122/tile"
output_path = "calib_result"

with open("calib_result/calib_result.yaml") as f:
    calib = yaml.safe_load(f)

k1 = np.array(calib["k2"])
d1 = np.array(calib["d2"])
# k2 = np.array(calib["cameraMatrix2"])
# d2 = np.array(calib["distCoeffs2"])
R = np.array(calib["R"])
T = np.array(calib["T"])
RT = np.hstack((R, T))

In [3]:
# load zivid point cloud
with zivid.Application() as app:
    frame = zivid.Frame(glob(os.path.join(calib_path, "*.zdf"))[0])
    point_cloud = frame.point_cloud()
    xyz = point_cloud.copy_data("xyz")
    rgba = point_cloud.copy_data("rgba")
    rgb = rgba[..., :3]

print("xyz.shape:", xyz.shape)
print("rgb.shape:", rgb.shape)

xyz.shape: (1200, 1920, 3)
rgb.shape: (1200, 1920, 3)


In [4]:
calib_fuji_file = glob(os.path.join(calib_path, "*.RAF"))
fuji_rgb1, fuji_rgb2 = load_fuji_rgb(calib_fuji_file[0]), load_fuji_rgb(calib_fuji_file[1])
fuji_rgb = np.max(np.stack([fuji_rgb1, fuji_rgb2], axis=0), axis=0)

fuji_rgb_8bit = convert_16bit_to_8bit(fuji_rgb)

# rotate xyz 180 degrees
xyz_ = xyz.copy()
xyz_[:, :, :2] = np.negative(xyz_[:, :, :2])
xyz_ = xyz_[:, 160:-160, :]
rgb_ = rgb[:, 160:-160, :]

# remove NaNs in xyz
xyz_ = xyz_.reshape(-1, 3)
rgb_= rgb_.reshape(-1, 3)

rgb_ = rgb_[~np.isnan(xyz_).any(axis=1)]
xyz_ = xyz_[~np.isnan(xyz_).any(axis=1)]

# # preview point cloud
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(xyz_)
# pcd.colors = o3d.utility.Vector3dVector(rgb_ / 255)
# o3d.visualization.draw_geometries([pcd])

Load fuji img with shape (8733, 11644, 3)
Load fuji img with shape (8733, 11644, 3)


In [5]:
# project xyz to img_f with k1, d1 and RT
uv_ = []
for i in trange(len(xyz_)):
    uv = cv2.projectPoints(xyz_[i], R, T, k1, d1)[0]
    uv_.append(uv)

uv_ = np.array(uv_).reshape(-1, 2)

100%|██████████| 1875232/1875232 [00:20<00:00, 91512.17it/s]


In [6]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_)

# fill the color of the point cloud with fuji_rgb_8bit values at uv_
pcd.colors = o3d.utility.Vector3dVector(fuji_rgb_8bit[uv_[:, 1].astype(int), uv_[:, 0].astype(int), :]/255)
# save pcd to ply
o3d.io.write_point_cloud(os.path.join(output_path, "pcd.ply"), pcd)
# visualize pcd
# o3d.visualization.draw_geometries([pcd])

True